In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('missing').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/03 04:01:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
wash_df = spark.read.csv("WASH.csv",header=True)
death_df = spark.read.csv("unsafewaterdeathrate.csv",header=True)
gdp_df = spark.read.csv("gdp-per-capita-worldbank.csv",header=True)

In [5]:
from pyspark.sql.types import (StructField,StringType,IntegerType,FloatType,StructType)
data_schema = [StructField('Entity',StringType(),True),
              StructField('Code',StringType(),True),
              StructField('Year',IntegerType(),True),
              StructField('Share of total deaths that are from all causes attributed to unsafe water source, in both sexes aged age-standardized',FloatType(),True)]
final_struct = StructType(fields=data_schema)
death_df = spark.read.csv("unsafewaterdeathrate.csv",schema=final_struct,header=True)
death_df.printSchema()

data_schema = [StructField('Entity',StringType(),True),
              StructField('Code',StringType(),True),
              StructField('Year',IntegerType(),True),
              StructField('GDP per capita, PPP (constant 2017 international $)',FloatType(),True)]
final_struct = StructType(fields=data_schema)
gdp_df = spark.read.csv("gdp-per-capita-worldbank.csv",schema=final_struct,header=True)
gdp_df.printSchema()

data_schema = [StructField('Type',StringType(),True),
              StructField('Region',StringType(),True),
              StructField("Residence Type",StringType(),True),
              StructField("Service Type",StringType(),True),
              StructField('Year',IntegerType(),True),
              StructField('Coverage',FloatType(),True),
              StructField('Population',FloatType(),True),
              StructField('Facility type',StringType(),True)]
final_struct = StructType(fields=data_schema)
wash_df = spark.read.csv("WASH.csv",schema=final_struct,header=True)
wash_df.printSchema()

root
 |-- Entity: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Share of total deaths that are from all causes attributed to unsafe water source, in both sexes aged age-standardized: float (nullable = true)

root
 |-- Entity: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- GDP per capita, PPP (constant 2017 international $): float (nullable = true)

root
 |-- Type: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Residence Type: string (nullable = true)
 |-- Service Type: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Coverage: float (nullable = true)
 |-- Population: float (nullable = true)
 |-- Facility type: string (nullable = true)



## Select Target variables for our analysis


In [12]:
# drop the 'Type' column
wash_df = wash_df.drop('Type')
wash_df.describe().show()

+-----------+--------------+--------------+----+--------+------------+--------------------+
|     Region|Residence Type|  Service Type|Year|Coverage|  Population|       Facility type|
+-----------+--------------+--------------+----+--------+------------+--------------------+
|High income|         total|    Sanitation|2012| 3.16338|  3.703898E7|Improved latrine ...|
|High income|         rural|    Sanitation|2012| 7.33999| 1.7285014E7|Improved latrine ...|
|High income|         urban|    Sanitation|2012|  1.8344|  1.715856E7|Improved latrine ...|
|High income|         total|    Sanitation|2012|13.44265| 1.5739536E8|         Septic tank|
|High income|         rural|    Sanitation|2012|33.11531| 7.7983552E7|         Septic tank|
|High income|         urban|    Sanitation|2012| 7.74518| 7.2446424E7|         Septic tank|
|High income|         total|    Sanitation|2012|82.65761| 9.6780966E8|               Sewer|
|High income|         rural|    Sanitation|2012|57.11532|1.34501392E8|          

+-------+-------------------+--------------+--------------+-----------------+-----------------+--------------------+--------------------+
|summary|             Region|Residence Type|  Service Type|             Year|         Coverage|          Population|       Facility type|
+-------+-------------------+--------------+--------------+-----------------+-----------------+--------------------+--------------------+
|  count|                600|           600|           600|              600|              600|                 600|                 600|
|   mean|               null|          null|          null|           2016.5|33.61109341591597|4.3656411323833334E8|                null|
| stddev|               null|          null|          null|2.874677887159652|27.21752830198907| 4.782407787147155E8|                null|
|    min|        High income|         rural|Drinking water|             2012|          0.78282|           7231180.5|Improved latrine ...|
|    max|Upper middle income|     

In [13]:
# drop the 'Code' column
death_df = death_df.drop('Code')
death_df.describe().show()

+-------+-----------+-----------------+---------------------------------------------------------------------------------------------------------------------+
|summary|     Entity|             Year|Share of total deaths that are from all causes attributed to unsafe water source, in both sexes aged age-standardized|
+-------+-----------+-----------------+---------------------------------------------------------------------------------------------------------------------+
|  count|       6840|             6840|                                                                                                                 6840|
|   mean|       null|           2004.5|                                                                                                   2.2082172846679153|
| stddev|       null|8.656074225491706|                                                                                                     3.09963769859107|
|    min|Afghanistan|             1990|             